In [1]:
import torch
import sys
import gc
sys.path.append('..')
%load_ext autoreload
%autoreload 2
from trl_wrapper.trainer_wrapper import TrainerWrapper, LLAMA_CONFIG, SMOL_LM_135M

/admin/home-brianf/smolmodels/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from dataclasses import replace
cfg = replace(LLAMA_CONFIG, model_id_or_path=SMOL_LM_135M, max_samples=100, single_process_mode=True)
wrapper = TrainerWrapper(cfg)
wrapper.init_model()

2024-12-18 16:16:27.056 | INFO     | trl_wrapper.trainer_wrapper:init_model:88 - Loading model HuggingFaceTB/SmolLM2-135M-Instruct
The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`


In [3]:
wrapper.init_data_module()

2024-12-18 16:18:18.242 | INFO     | model.utils:setup:101 - Loading dataset for stage fit
2024-12-18 16:18:18.244 | INFO     | model.utils:setup:103 - Processing dataset for stage fit, workers: 1, cache dir dataset_caches/ultrafeedback
2024-12-18 16:18:20.243 | INFO     | dataset.squad:load_dataset:248 - Loaded dataset with 60917 samples
2024-12-18 16:18:20.349 | WARNING  | model.utils:setup:144 - Computed columns not found in dataset: ['source', 'prompt', 'chosen', 'chosen-rating', 'chosen-model', 'rejected', 'rejected-rating', 'rejected-model'] assuming using 


In [ ]:
gc.collect()
torch.cuda.empty_cache()
! echo $CUDA_VISIBLE_DEVICES
! nvidia-smi

In [14]:
wrapper.init_trainer()

2024-12-18 16:29:41.250 | INFO     | trl_wrapper.trainer_wrapper:init_trainer:140 - LoRA already loaded, ignoring
2024-12-18 16:29:41.440 | INFO     | trl_wrapper.trainer_wrapper:init_trainer:191 - Initializing DPOTrainer, with project name: codecontests-llama-3b, run_name: run-390448, logprobs cache: dataset_caches/ultrafeedback/b68b303b/ref_logprobs_cache peft config: LoraConfig(task_type='CAUSAL_LM', peft_type=<PeftType.LORA: 'LORA'>, auto_mapping=None, base_model_name_or_path=None, revision=None, inference_mode=False, r=512, target_modules='dummy-target-modules', exclude_modules=None, lora_alpha=256, lora_dropout=0.2, fan_in_fan_out=False, bias='none', use_rslora=False, modules_to_save=None, init_lora_weights=True, layers_to_transform=None, layers_pattern=None, rank_pattern={}, alpha_pattern={}, megatron_config=None, megatron_core='megatron.core', loftq_config={}, eva_config=None, use_dora=False, layer_replication=None, runtime_config=LoraRuntimeConfig(ephemeral_gpu_offload=False),

In [17]:
wrapper.train()

2024-12-18 16:36:51.712 | INFO     | trl_wrapper.trainer_wrapper:train:318 - Starting training.
2024-12-18 16:36:52.201 | INFO     | trl_wrapper.dpo_trainer:evaluation_loop:134 - Generating samples...
2024-12-18 16:36:52.201 | INFO     | trl_wrapper.dpo_trainer:generate_from_model_and_ref:56 - Generating policy samples, max length: 512...
2024-12-18 16:36:57.276 | INFO     | trl_wrapper.dpo_trainer:generate_from_model_and_ref:74 - Generating reference samples, max length: 512...
2024-12-18 16:37:07.352 | INFO     | trl_wrapper.dpo_trainer:evaluation_loop:200 - Saved eval samples.


Step,Training Loss,Validation Loss,Rewards/chosen,Rewards/rejected,Rewards/accuracies,Rewards/margins,Logps/chosen,Logps/rejected,Logits/chosen,Logits/rejected
0,No log,0.871759,-7.583045,-9.695808,0.700000,2.112763,-673.983154,-519.704956,-0.110224,-0.599418
100,0.001400,0.722615,-7.104337,-9.703661,0.730000,2.599325,-669.196106,-519.783447,0.521999,0.074824


2024-12-18 16:37:53.074 | INFO     | trl_wrapper.dpo_trainer:evaluation_loop:134 - Generating samples...
2024-12-18 16:37:53.074 | INFO     | trl_wrapper.dpo_trainer:generate_from_model_and_ref:56 - Generating policy samples, max length: 512...
2024-12-18 16:37:58.123 | INFO     | trl_wrapper.dpo_trainer:generate_from_model_and_ref:74 - Generating reference samples, max length: 512...
2024-12-18 16:38:08.216 | INFO     | trl_wrapper.dpo_trainer:evaluation_loop:200 - Saved eval samples.


KeyboardInterrupt: 

In [ ]:
from IPython.display import Markdown
for prompt, chosen, rejected, logprob_diff in []:
    display(Markdown(f"\n### Prompt: {prompt}\n\n### Chosen:\n {chosen}\n\n### Rejected:\n {rejected}\n\nLogprob diff: {logprob_diff}"))